# Fine Tuning XLNet Model for Text Classification

### Download the data from Kaggle: 
 - https://www.kaggle.com/c/nlp-getting-started/data
 
In this competition, you’re challenged to build a machine learning model that predicts which Tweets are about real disasters and which one’s aren’t. You’ll have access to a dataset of 10,000 tweets that were hand classified. 

In [2]:
import pandas as pd
import numpy as np

In [3]:
%%capture
!pip install wandb

In [4]:
import wandb

wandb.login()

wandb: Currently logged in as: huma- (use `wandb login --relogin` to force relogin)


True

In [6]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [7]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
df_train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
df_train.keyword.isnull().sum()/df_train.shape[0]*100

0.8012610009194797

In [9]:
df_train.location.isnull().sum()/df_train.shape[0]*100

33.27203467752528

In [11]:
df_train.sample(10)['text'].tolist()

['Byproduct of metal price meltdown is a higher silver price http://t.co/cZWjw4UV7i',
 'Tension In Bayelsa As Patience Jonathan Plans To Hijack APC PDP - http://t.co/NIpZmfLiBD',
 '#DroughtMonitor: Moderate or worse #drought ? to ~27% of contig USA; affects ~80M people. http://t.co/YBE9JQoznR http://t.co/328SzflEtZ',
 'I spent 15 minutes lifting weights. 43 calories burned. #LoseIt',
 'Texas Seeks Comment on Rules for Changes to Windstorm Insurer http://t.co/BNNIdfZWbd',
 "Ted Cruz Bashes Obama Comparison GOP To Iranians Shouting 'Death To America' http://t.co/tXETcysm1H | #tcot",
 'Consent Order on cleanup underway at CSX derailment site - Knoxville News Sentinel http://t.co/GieSoMgWTR http://t.co/NMFsgKf1Za',
 'Haley Lu Richardson Fights for Water in The Last Survivors (Review) http://t.co/oObSCFOKtQ',
 'Meet the man who survived both Hiroshima and Nagasaki http://t.co/PNSsIa5e46 http://t.co/LSVsYSpdxX',
 "@Re_ShrimpLevy I'm always up for mass murder"]

## Cleaning
 - Replace `#`
 - Remove username starting with `@`
 - Remove `links`

In [12]:
! pip install tweet-preprocessor
import preprocessor as p

def clean_text(text):
  text = text.replace("#","")
  return p.clean(text)

In [13]:
from tqdm.notebook import tqdm
tqdm.pandas()

df_train['clean_text'] = df_train['text'].astype(str).progress_map(clean_text)
df_test['clean_text'] = df_test['text'].astype(str).progress_map(clean_text)

  0%|          | 0/7613 [00:00<?, ?it/s]

  0%|          | 0/3263 [00:00<?, ?it/s]

In [14]:
# splitting the data into training and test dataset
X = df_train['clean_text']
y = df_train['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [15]:
train_df = pd.DataFrame(X_train)
train_df['target'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['target'] = y_test

In [16]:
train_df.shape, eval_df.shape

((6090, 2), (1523, 2))

In [17]:
# transformers - SOTA implementation of pretrained models
!pip install -U simpletransformers 

In [28]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
'''
args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 256,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 3,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,"distilbert":
    model_name = "distilbert-base-cased"
    elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"
}

'''

# Create a ClassificationModel
model = ClassificationModel("electra", "google/electra-base-discriminator", args={'num_train_epochs':10, 'train_batch_size':32, 'max_seq_length':128,"wandb_project": "electrasimple",
    "wandb_kwargs": {"name": "google/electra-base-discriminator"},'overwrite_output_dir': True,}) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

  0%|          | 0/6090 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_electra_128_2_2


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


...

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_electra_128_2_2


Running Evaluation:   0%|          | 0/191 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,█▅▇▅▆▇▅▄▅▄▅▄▄▂▄▂▂▆▃▄▂▃▂▁▃▃▂▂▂▃▁▃▃▁▂▁▂▂
global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,▄▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
Training loss,0.05689
global_step,1900
lr,0.0


INFO:simpletransformers.classification.classification_model:{'mcc': 0.5360512821973193, 'tp': 479, 'tn': 698, 'fp': 171, 'fn': 175, 'auroc': 0.8406337207870132, 'auprc': 0.8001330160681145, 'acc': 0.7728168089297439, 'eval_loss': 0.9153334751179081}


In [29]:
result

{'mcc': 0.5360512821973193,
 'tp': 479,
 'tn': 698,
 'fp': 171,
 'fn': 175,
 'auroc': 0.8406337207870132,
 'auprc': 0.8001330160681145,
 'acc': 0.7728168089297439,
 'eval_loss': 0.9153334751179081}

In [30]:
predictions, raw_outputs = model.predict(df_test.clean_text.tolist())

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3263 [00:00<?, ?it/s]

  0%|          | 0/408 [00:00<?, ?it/s]

In [31]:
sample_sub=pd.read_csv("sample_submission.csv")
sample_sub['target'] = predictions

sample_sub.to_csv("submission_electra_base.csv", index=False)